In [3]:
%load_ext autoreload
%autoreload 2

import os, sys, re, datetime, random, gzip, json, copy
from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from itertools import accumulate
import argparse
from time import time
from math import ceil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.trainer.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.utilities.seed import seed_everything

from torch_geometric.loader import DataLoader
from torch_geometric.datasets import UPFD
from torch_geometric.nn import Sequential, HeteroConv, GINConv, GCNConv, SAGEConv, GATConv

PROJ_PATH = Path(os.path.join(re.sub("/UPFD_ATT.*$", '', os.getcwd()), 'UPFD_ATT'))
print(f'PROJ_PATH={PROJ_PATH}')
sys.path.insert(1, str(PROJ_PATH))
sys.path.insert(1, str(PROJ_PATH/'src'))
import utils
from utils import *
from dataset import DataModule
from trainer import build_trainer
from model import GNN, GNN_ATT

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PROJ_PATH=/home/hminle/github/UPFD_ATT


# Prepare data

In [4]:
train_data = UPFD(root="./data", name="gossipcop", feature="content", split="train")
val_data = UPFD(root="./data", name="gossipcop", feature="content", split="val")
test_data = UPFD(root="./data", name="gossipcop", feature="content", split="test")

Extracting data/gossipcop/raw/uc
Processing...
Done!


In [5]:
print(len(train_data), len(val_data), len(test_data))

1092 546 3826


In [7]:
train_data = UPFD(root="./data", name="politifact", feature="content", split="train")
val_data = UPFD(root="./data", name="politifact", feature="content", split="val")
test_data = UPFD(root="./data", name="politifact", feature="content", split="test")

Extracting data/politifact/raw/uc
Processing...
Done!


In [8]:
print(len(train_data), len(val_data), len(test_data))

62 31 221


# Train/Test/Inference

In [2]:
parser = argparse.ArgumentParser(description='Training.')
parser.add_argument('-config_file', help='config file path', default=str(PROJ_PATH / 'src/config/gnn_gos.json'), type=str)
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
args = parser.parse_args()
print('Reading config file:', args.config_file)
args.config = utils.read_json(args.config_file)
seed_everything(args.config['trainer_params']['seed'], workers=True)

Global seed set to 2022


Reading config file: /home/hminle/github/UPFD_ATT/src/config/gnn_gos.json


2022

In [3]:
args.config

OrderedDict([('data_params',
              OrderedDict([('root', '../data'),
                           ('name', 'gossipcop'),
                           ('feature', 'content'),
                           ('batch_size', 128)])),
             ('model_params',
              OrderedDict([('model_name', 'GNN'),
                           ('in_channels', 310),
                           ('hidden_channels', 128),
                           ('out_channels', 1),
                           ('lr', 0.001)])),
             ('trainer_params',
              OrderedDict([('seed', 2022),
                           ('name', 'gnn_gos'),
                           ('checkpoint_dir', '../model'),
                           ('top_k', 5),
                           ('max_epochs', 200),
                           ('metric', 'val_acc'),
                           ('patience', 20),
                           ('mode', 'max')]))])

In [4]:
# Train
data_module = DataModule(args.config['data_params'])
if args.config['model_params']['model_name'] == 'GNN':
    model_module = GNN(args.config['model_params'])
elif args.config['model_params']['model_name'] == 'GNN_ATT':
    model_module = GNN_ATT(args.config['model_params'])
trainer, _ = build_trainer(args.config['trainer_params'])
data_module.setup()
trainer.fit(model_module, data_module)

/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:192: LightningDeprecationWarning: Setting `Trainer(weights_summary=full)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.model_summary.ModelSummary` with `max_depth` directly to the Trainer's `callbacks` argument instead.
  f"Setting `Trainer(weights_summary={weights_summary})` is deprecated in v1.5 and will be removed"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /home/hminle/github/UPFD_ATT/model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name          | Type    | Pa

Sanity Checking: 0it [00:00, ?it/s]

/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7196. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/callbacks/progress/base.py:223: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  f"The progress bar already tracks a metric with the name(s) '{', '.join(duplicates)}' and"
Epoch 26, global step 216: 'val_acc' was not in top 5


Validation: 0it [00:00, ?it/s]

/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/callbacks/progress/base.py:223: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  f"The progress bar already tracks a metric with the name(s) '{', '.join(duplicates)}' and"
Epoch 27, global step 224: 'val_acc' was not in top 5


Validation: 0it [00:00, ?it/s]

/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/callbacks/progress/base.py:223: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  f"The progress bar already tracks a metric with the name(s) '{', '.join(duplicates)}' and"
Epoch 28, global step 232: 'val_acc' was not in top 5
/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [6]:
# Test
print('### Test')
checkpoint_dir = Path(args.config['trainer_params']['checkpoint_dir'])
print(f'Load checkpoint from: {str(checkpoint_dir)}')
paths = sorted(checkpoint_dir.glob('*.ckpt'))
name = args.config['trainer_params']['name']
filtered_paths = [p for p in paths if f'model={name}-' in str(p)]
results = []
for i, p in enumerate(filtered_paths):
    print(f'Load model {i}: {p}')
    # test
    model_test = model_module.load_from_checkpoint(checkpoint_path=p) 
    result = trainer.test(model_test, datamodule=data_module)
    results.append(results)
    del model_test

### Test
Load checkpoint from: ../model
Load model 0: ../model/model=gnn_gos-epoch=10-val_loss=0.1142-val_acc=0.9750-val_f1=0.9760.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9730491593567251
         test_f1            0.9731425835991645
        test_loss           0.10262882709503174
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Load model 1: ../model/model=gnn_gos-epoch=14-val_loss=0.1017-val_acc=0.9766-val_f1=0.9773.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


...

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9772797880116959
         test_f1            0.9774512809093417
        test_loss           0.08736326545476913
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Load model 9: ../model/model=gnn_gos-epoch=24-val_loss=0.1186-val_acc=0.9750-val_f1=0.9759.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/hminle/miniconda3/envs/upfd/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9738304093567252
         test_f1             0.974019319519677
        test_loss           0.10413514822721481
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [19]:
# inference
model_path = '../model/model=gnn_gos-epoch=10-val_loss=0.1142-val_acc=0.9750-val_f1=0.9760.ckpt'
device = 'cuda'
map_location = lambda storage, loc: storage.cuda()
checkpoint = torch.load(model_path, map_location=map_location)
model_dict = model_module.state_dict()
pretrain_dict = {k: v for k, v in checkpoint['state_dict'].items() if k in model_dict}
model_module.load_state_dict(pretrain_dict)
model_module.eval()
model_module.to(device)

GNN(
  (conv1): GATConv(310, 128, heads=1)
  (conv2): GATConv(128, 128, heads=1)
  (conv3): GATConv(128, 128, heads=1)
  (lin_news): Linear(in_features=310, out_features=128, bias=True)
  (lin0): Linear(in_features=128, out_features=128, bias=True)
  (lin1): Linear(in_features=256, out_features=1, bias=True)
  (loss): BCELoss()
)

In [34]:
pred_logits = []
preds = []
trues = []
for batch in data_module.test_dataloader():
    pred = model_module.forward(
        batch['x'].to(device), 
        batch['edge_index'].to(device), 
        batch['batch'].to(device))
    pred_logits += list(pred.cpu().detach().numpy()[:,0])
    preds += list(torch.round(pred).cpu().detach().numpy()[:,0])
    trues += list(batch.y.cpu().numpy())
df_result = pd.DataFrame({
    'pred_logits': pred_logits,
    'preds': preds,
    'trues': trues,
})
df_result.head(20)

,pred_logits,preds,trues
0,0.997363,1.0,1
1,0.956657,1.0,1
2,0.128067,0.0,0
3,0.968894,1.0,1
4,0.031830,0.0,0
5,0.037398,0.0,0
6,0.031676,0.0,0
7,0.997774,1.0,1
8,0.028810,0.0,0
9,0.038423,0.0,0
